In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tqdm
import pickle
import os

from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

In [2]:
def calc_dnml(X, Z1, Z2, K=3, L=3):    
    N = X.shape[0]
    
    codelen_x_z = 0.0
    codelen_z = 0.0

    for k in range(K):
        for l in range(L):
            lam = np.mean(X[Z1 == k, :][:, Z2 == l])

            codelen_x_z += n_all * np.log(n_all)
            codelen_x_z += np.log(normterm_discrete(n_all, 2))

        n_k = np.sum(Z1 == k)
        if n_k >= 1:
            codelen_z += n_k * (np.log(N) - np.log(n_k))

    codelen_z += np.log(normterm_discrete(N, K))
    
    codelen = codelen_x_z + codelen_z

    return codelen, codelen_x_z, codelen_z

In [3]:
outdir = 'output/'
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [4]:
blockmodels = importr("blockmodels")
base = importr("base")
dollar = base.__dict__["$"]

In [5]:
EPS = np.finfo(np.float).eps

In [6]:
# read data
d = pd.read_csv('data_processed/FEH_200504_201503.csv')

In [7]:
d.head(3)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,北海道,青森県,岩手県,宮城県,秋田県,山形県,...,愛媛県,高知県,福岡県,佐賀県,長崎県,熊本県,大分県,宮崎県,鹿児島県,沖縄県
0,2015000303,2015年3月,1000,北海道,-,400,146,472,128,102,...,49,31,304,24,48,70,30,29,39,129
1,2015000303,2015年3月,2000,青森県,535,-,536,942,222,95,...,2,1,41,7,27,17,12,3,15,139
2,2015000303,2015年3月,3000,岩手県,231,421,-,"1,309",215,122,...,2,8,33,2,4,6,4,4,9,8


In [8]:
columns = d.loc[:, '北海道':'沖縄県'].columns

In [9]:
columns

Index(['北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県', '茨城県', '栃木県', '群馬県',
       '埼玉県', '千葉県', '東京都', '神奈川県', '新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県',
       '岐阜県', '静岡県', '愛知県', '三重県', '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県',
       '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県', '香川県', '愛媛県', '高知県', '福岡県',
       '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県'],
      dtype='object')

In [10]:
for cn in columns:
    d[cn] = d[cn].str.replace('-', '0').str.replace(',', '').astype(np.int)

In [11]:
YM_list = np.unique(d['Unnamed: 0'].values)

In [12]:
with open(os.path.join(outdir, 'ym_list.pkl'), 'wb') as f:
    pickle.dump(YM_list, f)

In [13]:
d_list = []
for ym in YM_list:
    d_ym = d.loc[d['Unnamed: 0'] == ym, :]
    d_ym_values = d_ym.iloc[:, 4:].values
    d_ym_values = d_ym_values.astype(np.float)
    d_ym_values[np.isnan(d_ym_values)] = 0.0
    d_ym_values = d_ym_values.astype(np.int)
    
    d_list.append(d_ym_values)

In [14]:
d_list_array = np.array(d_list)

In [15]:
with open('data_processed/X.pkl', 'wb') as f:
    pickle.dump(d_list_array, f)

In [16]:
def estimate_poisson(X, K):
    numpy2ri.activate()
    
    sbm = blockmodels.BM_poisson(membership_type="SBM", adj=np.array(X),
                                 verbosity=0,
                                 exploration_factor=1.5,
                                 explore_min=K,
                                 explore_max=K)

    estimate = dollar(sbm, "estimate")
    estimate()

    pi_list = []
    theta_list = []
    z_posterior_list = []
    
    for k in range(K):
        n_clusters = k + 1
        theta = np.array(dollar(dollar(sbm, "model_parameters")[k], "lambda"))
        z_posterior = np.array(dollar(dollar(sbm, "memberships")[k], "Z"))
        pi = np.sum(z_posterior, axis=0) + 10 * EPS
        pi /= np.sum(pi)

        theta_list.append(theta)
        z_posterior_list.append(z_posterior)
        pi_list.append(pi)

    numpy2ri.deactivate()

    return pi_list, theta_list, z_posterior_list

In [17]:
T = len(YM_list)
h = 2
K = 10

theta_list = [] 
pi_list = []
z_list = []

for t in tqdm.tqdm(range(T)):
    d = d_list_array[t, :, :]
    
    pi, theta, z = estimate_poisson(d, K=K)
    pi_list.append(pi)
    theta_list.append(theta)
    z_list.append(z)

  0%|          | 0/120 [00:00<?, ?it/s]

  1%|          | 1/120 [00:02<05:19,  2.69s/it]

  2%|▏         | 2/120 [00:06<05:42,  2.90s/it]

  2%|▎         | 3/120 [00:08<05:38,  2.89s/it]

  3%|▎         | 4/120 [00:11<05:18,  2.75s/it]

...

 97%|█████████▋| 116/120 [06:01<00:14,  3.51s/it]

 98%|█████████▊| 117/120 [06:04<00:09,  3.28s/it]

 98%|█████████▊| 118/120 [06:07<00:06,  3.21s/it]

 99%|█████████▉| 119/120 [06:09<00:03,  3.07s/it]

100%|██████████| 120/120 [06:12<00:00,  3.02s/it]


In [18]:
with open(os.path.join(outdir, 'pi_list.pkl'), 'wb') as f:
    pickle.dump(pi_list, f)

with open(os.path.join(outdir, 'theta_list.pkl'), 'wb') as f:
    pickle.dump(theta_list, f)
    
with open(os.path.join(outdir, 'z_list.pkl'), 'wb') as f:
    pickle.dump(z_list, f)